In [3]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

# Function to load images and preprocess data
def load_data(data_dir, image_height, image_width):
    images = []
    tread_depths = []

    for subdir, _, files in os.walk(data_dir):
        for file in files:
            if file.endswith('.jpg'):
                # Read image
                image_path = os.path.join(subdir, file)
                image = cv2.imread(image_path)
                image = cv2.resize(image, (image_width, image_height))
                images.append(image)
                
                # Extract tread depth from filename or metadata
                tread_depth_str = ''.join(filter(str.isdigit, file))  # Extract only numerical characters
                tread_depth = float(tread_depth_str)  # Convert to float
                tread_depths.append(tread_depth)

    # Convert lists to numpy arrays
    images = np.array(images)
    tread_depths = np.array(tread_depths)

    return images, tread_depths

# Load data
data_dir = '/kaggle/input/data707'  # Replace with the path to your dataset directory
image_height, image_width = 100, 100  # Specify image dimensions
images, tread_depths = load_data(data_dir, image_height, image_width)

# Split data into training and validation sets
train_images, val_images, train_tread_depths, val_tread_depths = train_test_split(
    images, tread_depths, test_size=0.2, random_state=42)

# Define the model architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Output layer for tread depth prediction
])

# Compile the model
model.compile(optimizer='adam',
              loss='mean_squared_error',  # Use MSE loss for regression
              metrics=['mae'])  # Mean Absolute Error as evaluation metric

# Train the model
history = model.fit(train_images, train_tread_depths,
                    epochs=10,
                    validation_data=(val_images, val_tread_depths))

# Evaluate the model
val_loss, val_mae = model.evaluate(val_images, val_tread_depths)
print("Validation Loss:", val_loss)
print("Validation MAE:", val_mae)

# Save the model
model.save('tread_depth_model.h5')


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 3427.8572 - mae: 43.9179 - val_loss: 209355.2344 - val_mae: 457.5535
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step - loss: 276963.1562 - mae: 517.8609 - val_loss: 6992.4517 - val_mae: 83.6209
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - loss: 2941.7275 - mae: 45.6563 - val_loss: 19415.0566 - val_mae: 139.3379
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - loss: 10121.8535 - mae: 95.8539 - val_loss: 12098.4395 - val_mae: 109.9929
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - loss: 5565.3965 - mae: 67.5749 - val_loss: 6931.5825 - val_mae: 83.2561
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - loss: 2772.0996 - mae: 41.6505 - val_loss: 3692.0073 - val_mae: 60.7619
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 1470.0422 - mae: 34.8290 - val_loss: 1845.0208 - val_mae: 42.9537
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 1040.1826 - mae: 32.1028 - val_loss: 1774.7152 - val_mae: 42.1274
Epo

In [4]:
# Evaluate the model
val_loss, val_mae = model.evaluate(val_images, val_tread_depths)
print("Validation Loss:", val_loss)
print("Validation MAE:", val_mae)

# Calculate accuracy (lower MAE is better)
accuracy = 1 - (val_mae / np.mean(val_tread_depths))
print("Accuracy:", accuracy)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 2.1305 - mae: 1.4596
Validation Loss: 2.1304612159729004
Validation MAE: 1.4596099853515625
Accuracy: 0.9828281178193934
